# Filtro de Dragón que escupe fuego cuando se abre la boca


In [1]:
import cv2
import dlib
import numpy as np
import math

# Cargar el predictor de puntos faciales
predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()  # Detector de rostros
predictor = dlib.shape_predictor(predictor_path)  # Predictor de puntos faciales

# Cargar la máscara de dragón y la imagen de fuego
mask = cv2.imread("dragon_mask_1.png", cv2.IMREAD_UNCHANGED)  # Máscara de dragón
fire_effect = cv2.imread("fire.png", cv2.IMREAD_UNCHANGED)  # Imagen de efecto de fuego

# Configuración inicial para el efecto de fuego
fire_positions = []  # Lista para almacenar posiciones del fuego
fire_length_increment = 10  # Incremento de longitud entre llamas
max_fire_length = 200  # Longitud máxima del fuego
fire_emoji_size = (32, 32)  # Tamaño de cada "emoji" de fuego

fire_angle = 0  # Ángulo inicial del fuego

# Función para verificar si la boca está abierta
def is_mouth_open(shape):
    top_lip = shape.part(51).y  # Punto superior del labio
    bottom_lip = shape.part(57).y  # Punto inferior del labio
    lip_distance = bottom_lip - top_lip  # Distancia entre los labios
    top_head = shape.part(27).y
    bottom_head = shape.part(8).y
    head_distance = bottom_head - top_head

    return lip_distance/head_distance > 0.25

    # Se considera boca abierta si la distancia es mayor a 35 píxeles

# Función para superponer la máscara en el rostro detectado
def overlay_mask(frame, face, shape, mask):
    # Obtener puntos clave de la cara: barbilla (8), lado izquierdo (2) y lado derecho (14)
    chin = (shape.part(8).x, shape.part(8).y)
    left_side = (shape.part(2).x, shape.part(2).y)
    right_side = (shape.part(14).x, shape.part(14).y)

    # Calcular el centro de la cara y la distancia entre los lados izquierdo y derecho
    face_center = ((left_side[0] + right_side[0]) // 2, (left_side[1] + right_side[1]) // 2)
    face_width = int(np.linalg.norm(np.array(right_side) - np.array(left_side))) + 150

    # Ajuste vertical para posicionar la máscara más cerca de la nariz
    mask_position_y = (chin[1] + face_center[1]) // 2 - 120

    # Escalar la máscara según la anchura de la cara
    scale_factor = face_width / mask.shape[1]
    mask_resized = cv2.resize(mask, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
    mask_rgb = mask_resized[:, :, :3]  # Canal RGB de la máscara
    mask_alpha = mask_resized[:, :, 3] / 255.0  # Canal de transparencia de la máscara

    # Rotar la máscara para alinearla con el ángulo de la cara
    angle = math.degrees(math.atan2(right_side[1] - left_side[1], right_side[0] - left_side[0]))
    rotation_matrix = cv2.getRotationMatrix2D((mask_resized.shape[1] // 2, mask_resized.shape[0] // 2), angle, 1)
    mask_rgb = cv2.warpAffine(mask_rgb, rotation_matrix, (mask_resized.shape[1], mask_resized.shape[0]))
    mask_alpha = cv2.warpAffine(mask_alpha, rotation_matrix, (mask_resized.shape[1], mask_resized.shape[0]))

    # Calcular la posición de la máscara en la cara
    x_offset = face_center[0] - mask_rgb.shape[1] // 2
    y_offset = mask_position_y - mask_rgb.shape[0] // 2

    # Limitar la máscara dentro de los límites del marco
    y1, y2 = max(0, y_offset), min(y_offset + mask_rgb.shape[0], frame.shape[0])
    x1, x2 = max(0, x_offset), min(x_offset + mask_rgb.shape[1], frame.shape[1])

    # Ajustar la máscara y su transparencia al tamaño de la región de interés (ROI)
    mask_rgb = mask_rgb[:y2 - y1, :x2 - x1]
    mask_alpha = mask_alpha[:y2 - y1, :x2 - x1]
    roi = frame[y1:y2, x1:x2]

    # Aplicar la máscara en la región de interés usando el canal de transparencia
    for c in range(3):
        roi[:, :, c] = (mask_alpha * mask_rgb[:, :, c] + (1 - mask_alpha) * roi[:, :, c])

    frame[y1:y2, x1:x2] = roi  # Sobrescribir el marco original con el ROI
    return frame

# Función para superponer la secuencia de efectos de fuego
def overlay_fire_sequence(frame, fire_positions, fire_effect):
    # Redimensionar la imagen de fuego al tamaño del emoji de fuego
    fire_resized = cv2.resize(fire_effect, fire_emoji_size, interpolation=cv2.INTER_AREA)

    # Se normaliza el valor de transparecia del fuego para poder aplicarlo sobre el frame.
    fire_rgb = fire_resized[:, :, :3]
    fire_alpha = fire_resized[:, :, 3] / 255.0

    # Para cada posición de fuego almacenada, superponer la imagen de fuego en el marco
    for (x, y) in fire_positions:
        h, w = fire_resized.shape[:2]
        y1, y2 = max(0, y), min(y + h, frame.shape[0])
        x1, x2 = max(0, x - w // 2), min(x - w // 2 + w, frame.shape[1])

        if y2 <= y1 or x2 <= x1:
            continue

        # Ajustar el tamaño del fuego para que coincida con la región de interés (ROI)
        fire_rgb_resized = fire_rgb[:y2 - y1, :x2 - x1]
        fire_alpha_resized = fire_alpha[:y2 - y1, :x2 - x1]
        roi = frame[y1:y2, x1:x2]

        # Aplicar el efecto de fuego en la región de interés usando el canal de transparencia
        for c in range(3):
            roi[:, :, c] = (fire_alpha_resized * fire_rgb_resized[:, :, c] + (1 - fire_alpha_resized) * roi[:, :, c])

    return frame

# Inicializar captura de video
cap = cv2.VideoCapture(0)

# Bucle principal para procesar cada cuadro del video
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir el cuadro a escala de grises para detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    # Si se detectan rostros, procesar cada uno
    if len(faces) > 0:
        for face in faces:
            shape = predictor(gray, face)  # Obtener puntos faciales

            # Calcular el centro de la boca y la orientación
            left_mouth = (shape.part(48).x, shape.part(48).y)
            right_mouth = (shape.part(54).x, shape.part(54).y)
            mouth_center = ((left_mouth[0] + right_mouth[0]) // 2, (left_mouth[1] + right_mouth[1]) // 2)

            # Calcular la magnitud y el vector perpendicular para el ángulo del fuego. 
            # Con la magnitud, obtenida con la formula sqrt(dx^2 + dy^2), podemos obtener un vector perpendicular al establecido entre las diferencias de los laterales de la boca.

            dx = right_mouth[0] - left_mouth[0]     
            dy = right_mouth[1] - left_mouth[1]
            magnitude = np.sqrt(dx**2 + dy**2)

            # Obtenemos el vector perpendicular dividiendo la distancia euclidea (x2-x1, y2-y1) entre la magnitud calculada. EL valor de y es negativo porque el efecto es hacia abajo.

            perp_vector = (-dy / magnitude, dx / magnitude)     

            # Calcular el vector rotado para el ángulo actual del fuego
            rad_angle = math.radians(fire_angle)

            # Formula vector rotado -> x' = x*cos(a) - y*sin(b) ; y' = y*cos(a) + x*sin(b)

            rotated_vector = (
                perp_vector[0] * math.cos(rad_angle) - perp_vector[1] * math.sin(rad_angle),
                perp_vector[0] * math.sin(rad_angle) + perp_vector[1] * math.cos(rad_angle)
            )

            # Si la boca está abierta, agregar una nueva posición de fuego
            if is_mouth_open(shape):
                mouth_status = "Mouth: Open"

                # Nuevas posicion x e y del fuego

                new_fire_pos = (
                    int(mouth_center[0] + rotated_vector[0] * len(fire_positions) * fire_length_increment),
                    int(mouth_center[1] + rotated_vector[1] * len(fire_positions) * fire_length_increment)
                )
                fire_positions.append(new_fire_pos)

                # Limitar la longitud del fuego
                if len(fire_positions) * fire_length_increment > max_fire_length:
                    fire_positions.pop(0)
            
            # Si se cierra la boca, desaparece el fuego, se vacía la lista.
            else:
                mouth_status = "Mouth: Closed"
                fire_positions.clear()

            # Superponer la máscara de dragón y la secuencia de fuego
            frame = overlay_mask(frame, face, shape, mask)
            frame = overlay_fire_sequence(frame, fire_positions, fire_effect)

    else:
        mouth_status = "Mouth: Not detected"
        fire_positions.clear()

    # Mostrar el marco con el efecto aplicado
    cv2.imshow("Face with Dragon Mask and Fire Effect", frame)

    # Controlar el ángulo del fuego con teclas
    key = cv2.waitKey(1) & 0xFF
    if key == ord('a'):
        fire_angle -= 5
    elif key == ord('d'):
        fire_angle += 5
    elif key == ord('q'):
        break

# Liberar recursos y cerrar ventanas
cap.release()
cv2.destroyAllWindows()
